# ボルツマン変換に関する考察




## 1. 理論的背景  


ボルツマン変換とは物質などの1次元拡散流れの変換方法の一種である。  
この変換を用いることで物質移動の拡散係数が求められることが知られている。  

建築分野における一般的な説明は以下のように示される。  
まず、不飽和な材料中における水の水平1次元流れは以下のように表される。  

$$\frac{\partial \phi}{ \partial x} = \frac{\partial}{ \partial x} \Bigl (D(\phi) \frac{\partial \phi}{ \partial x} \Bigl ) $$  

ここで$\phi$は含水率[m3/m3]、$D(\phi)$は拡散係数[m2/s]である。  

なお、初期条件を乾燥状態とし、材料を片側から吸湿させていく過程を考えると、下記のような条件が必要となる。  
$ \phi(x,t) = \phi_i,  ( x\ge0, t = 0 )$   
$ \phi(x,t) = \phi_0,  ( x = 0, t > 0) $  
$ \phi(x,t) = \phi_i,  ( x → \infty, t > 0 )$

### ボルツマン変換より求められる拡散係数  

$$ D(w) = - \frac{1}{2} \frac{1}{\Bigl(\frac{d\phi}{d\lambda} \Bigl)_{\phi_{x}}} \int_{\phi_i}^{\phi_{x}} \lambda d\phi $$  

※計算式は後述

### 含水率-λ関係

$$\phi(\lambda) = - \Bigl[\phi_0 + a tan^{-1}(b\lambda +c) \Bigl]$$

In [1]:
function cal_phi_by_lam(phi0, a, b, c, lam)
    return - ( phi0 + a * atan( b*lam + c ) )
end
cal_phi_by_lam(;phi0, a, b, c, lam) = cal_phi_by_lam(phi0, a, b, c, lam)

cal_phi_by_lam (generic function with 2 methods)

In [2]:
cal_phi_by_lam(phi0 = 0.3, a = 0.43, b = 5.6, c = -8.6, lam = 1.2)

0.1652359081608739

### 逆関数（λ-含水率関係）

$$ \lambda(\phi) = \frac{1}{b} \Bigl[ tan \Bigl( - \frac{ \phi + \phi_{0} }{a} \Bigl)  - c  \Bigl]  $$

In [3]:
function cal_lam_by_phi(phi0, a, b, c, phi)
    return 1 / b * ( tan( - ( phi + phi0 ) / a ) -c )
end
cal_lam_by_phi(;phi0, a, b, c, phi) = cal_lam_by_phi(phi0, a, b, c, phi)

cal_lam_by_phi (generic function with 2 methods)

In [4]:
cal_lam_by_phi(phi0 = 0.3, a = 0.43, b = 5.6, c = -8.6, phi = cal_phi_by_lam(phi0 = 0.3, a = 0.43, b = 5.6, c = -8.6, lam = 1.2))

1.2

### 含水率のλ微分  

$$ \frac{\partial \phi}{\partial \lambda} = - \frac{ab}{1+(b\lambda+c)^2} $$

In [5]:
function cal_dphi_by_dlam(a, b, c, lam)
    return - a * b / ( 1 + ( b * lam + c )^2 )
end
cal_dphi_by_dlam(;a, b, c, lam) = cal_dphi_by_dlam(a, b, c, lam)

cal_dphi_by_dlam (generic function with 2 methods)

### λの含水率微分  

$$ \frac{\partial \lambda}{\partial \phi} = - \frac{1}{b  cos^{2} \Bigl( - \frac{\phi+\phi_{0}}{a}\Bigl) } $$

In [6]:
function cal_dlam_by_dphi(a, b, c, phi0, phi)
    return - 1 / ( b * cos( - ( phi + phi0 ) / a )^2 )
end
cal_dlam_by_dphi(;a, b, c, phi0, phi) = cal_dlam_by_dphi(a, b, c, phi0, phi)

cal_dlam_by_dphi (generic function with 2 methods)

### λの含水率積分

$$\int_{\phi_i}^{\phi_{x}} \lambda d\phi = - \frac{c}{b}(\phi_x-\phi_i) + \frac{a}{b}ln \Biggl( \frac{ |cos\frac{2\phi_x}{a}| }{ |cos\frac{\phi_x + \phi_i}{a}| } \Biggl)$$

In [7]:
function cal_int_lam_by_phi(a, b, c, phi_init, phi)
    return - c/b * ( phi - phi_init ) + a/b * log( abs(cos( 2*phi/a )) / abs( cos(( phi + phi_init )/a) ) )
end
cal_int_lam_by_phi(;a, b, c, phi_init, phi) = cal_int_lam_by_phi(a, b, c, phi_init, phi)

cal_int_lam_by_phi (generic function with 2 methods)

### 拡散係数  

$$ D(\phi) = - \frac{1}{2} \frac{1}{\Bigl(\frac{d\phi}{d\lambda} \Bigl)_{\phi_{x}}} \int_{\phi_i}^{\phi_{x}} \lambda d\phi $$  

In [11]:
function cal_Dw_by_Boltzmann( a, b, c, phi_init, phi)
    #lam     = cal_lam_by_phi(phi_0, a, b, c, phi)
    #dphi    = cal_dphi_by_dlam(a, b, c, lam)
    dlam    = cal_dlam_by_dphi(a, b, c, phi_init, phi)
    int_lam = cal_int_lam_by_phi(a, b, c, phi_init, phi)
    return - ( 1 / 2 ) * dlam * int_lam 
end
cal_Dw_by_Boltzmann(; a, b, c, phi_init, phi) = cal_Dw_by_Boltzmann( a, b, c, phi_init, phi)

cal_Dw_by_Boltzmann (generic function with 2 methods)

In [14]:
cal_Dw_by_Boltzmann( a = 0.43, b = 19400, c = -8.12, phi_init = 0.0, phi = 0.3)

4.0715681032037065e-9

### 水分伝導率$\lambda^{'}_{\mu}$への変換

含水率ベースの水分流$ j [m^{3}/ m^{2}s]$
$$ j = - D(\phi) \frac{\partial \phi}{\partial x} $$  

水分化学ポテンシャルベースの水分流$ J [kg/m^{2}s] $  
$$ J = - \lambda^{'}_{\mu} \frac{\partial \mu}{\partial x} $$  
$$ ⇔ J = - \lambda^{'}_{\mu} \frac{\partial \mu}{\partial \phi} \frac{\partial \phi}{\partial x} $$

水分伝導率の算出方法  
$$ \lambda^{'}_{\mu} = \rho_w D(\phi) \frac{\partial \phi}{\partial \mu} $$

In [15]:
function cal_ldml_by_Dw(Dw, dphi)
    rho_w = 1000.0
    return rho_w * Dw * dphi
end

cal_ldml_by_Dw (generic function with 1 method)